## Ads Library - Python

In [1]:
import facebook
import requests
import urllib3
import json
import pandas as pd
import csv

In [89]:
def main():
    token = "TOKEN"
    graph = facebook.GraphAPI(token)
    #define parameters for request
    profile = graph.get_object('ads_archive',fields='page_id,page_name,ad_snapshot_url,ad_creative_body,region_distribution,demographic_distribution',
                               search_terms='klimaat',
                               ad_type='POLITICAL_AND_ISSUE_ADS',
                               ad_reached_countries=['NL'])
    #return desired fields
    #print(json.dumps(profile, indent=4))
    #save results as json file
    with open('data.json', 'w', encoding='utf-8') as f:
        json.dump(profile, f, ensure_ascii=False, indent=4)

if __name__ == '__main__':
    main()
    
results = json.load(open('data.json'))
df = pd.DataFrame(results["data"])
df.to_csv('data.csv')

In [90]:
# Transform to ads table

df_ads = pd.DataFrame(results["data"])
df_ads = df_ads[['id','ad_creative_body','ad_snapshot_url','page_name','page_id']]

matrix = []
data = results["data"]
for i in range(len(data)):
    ids = data[i]['id']
    reg_dist = data[i]['region_distribution']
    for r in range(len(reg_dist)):
        perc = reg_dist[r]['percentage']
        reg = reg_dist[r]['region']
        matrix.append((ids,perc, reg))
df_reg_dist = pd.DataFrame(matrix).rename(columns={0:'id',1:'perc_region',2:'region'})

matrix = []
data = results["data"]
for i in range(len(data)):
    ids = data[i]['id']
    dem_dist = data[i]['demographic_distribution']
    for r in range(len(dem_dist)):
        perc = dem_dist[r]['percentage']
        age = dem_dist[r]['age']
        gen = dem_dist[r]['gender']
        matrix.append((ids, perc, age, gen))
df_dem_dist = pd.DataFrame(matrix).rename(columns={0:'id',1:'perc_demographic',2:'age',3:'gender'})

df_reg_dist = df_reg_dist.merge(df_ads, on='id',how='left')
df_reg_dist['perc_region'] = df_reg_dist['perc_region'].apply(float)
df_dem_dist = df_dem_dist.merge(df_ads, on='id',how='left')
df_dem_dist['perc_demographic'] = df_dem_dist['perc_demographic'].apply(float)

In [91]:
df_reg_dist.groupby('id').count()

,perc_region,region,ad_creative_body,ad_snapshot_url,page_name,page_id
id,,,,,,
1023754321480469,1,1,1,1,1,1
1034254717099339,12,12,12,12,12,12
175689150744699,1,1,1,1,1,1
1865817646933817,12,12,12,12,12,12
217052436841392,12,12,12,12,12,12
348735506562218,12,12,12,12,12,12
3740557662648493,12,12,12,12,12,12
422025382390044,12,12,12,12,12,12
780399602577376,12,12,12,12,12,12


In [92]:
df_reg_dist[df_reg_dist['id'] == '175689150744699']

,id,perc_region,region,ad_creative_body,ad_snapshot_url,page_name,page_id
61,175689150744699,1.0,Zuid-Holland,"Het klimaatakkoord van Parijs is een opdracht,...",https://www.facebook.com/ads/archive/render_ad...,CDA,320374518118


In [78]:
df_reg_dist[df_reg_dist['id'] == '1034254717099339'].sort_values('perc_region',ascending = False)

,id,perc_region,region,ad_creative_body,ad_snapshot_url,page_name,page_id
63,1034254717099339,0.264574,Noord-Holland,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
66,1034254717099339,0.251121,Zuid-Holland,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
68,1034254717099339,0.103139,Gelderland,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
70,1034254717099339,0.103139,North Brabant,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
64,1034254717099339,0.071749,Utrecht,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
69,1034254717099339,0.049327,Groningen,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
71,1034254717099339,0.049327,Overijssel,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
61,1034254717099339,0.040359,Drenthe,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
67,1034254717099339,0.026906,Flevoland,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
62,1034254717099339,0.022422,Limburg,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002


In [81]:
df_dem_dist[df_dem_dist['id'] == '1034254717099339'].sort_values('perc_demographic',ascending = False)

,id,perc_demographic,age,gender,ad_creative_body,ad_snapshot_url,page_name,page_id
74,1034254717099339,0.130045,25-34,female,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
73,1034254717099339,0.121076,35-44,male,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
82,1034254717099339,0.121076,25-34,male,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
75,1034254717099339,0.103139,35-44,female,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
78,1034254717099339,0.089686,45-54,male,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
71,1034254717099339,0.080717,18-24,female,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
77,1034254717099339,0.080717,45-54,female,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
83,1034254717099339,0.080717,55-64,male,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
79,1034254717099339,0.071749,18-24,male,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
81,1034254717099339,0.058296,55-64,female,💡 Wij willen kernenergie! Want het klimaat is ...,https://www.facebook.com/ads/archive/render_ad...,VVD,121264564551002
